In [1]:
import numpy as np
import math
import re
import subprocess
import pandas as pd
import signal
import scipy.constants as const
from scipy.stats import norm
from scipy.optimize import minimize

In [2]:
def to_fortran_double(number):
    if number == 0:
        return "0.000d0"
    abs_num = abs(number)
    sign = '-' if number < 0 else ""

    exponent = math.floor(math.log10(abs_num))
    mantissa = abs_num / (10 ** exponent)

    return f"{sign}{mantissa:.3f}d{exponent}"

In [3]:
def update_kinet_inp(file_path, fortran_doubles):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # 파라미터 라인을 찾아 수정
        pattern = re.compile(r'\$ double precision, parameter :: (f\d+)= .*')
        for i, line in enumerate(lines):
            match = pattern.match(line)
            if match:
                param_name = match.group(1)
                param_number = int(param_name[1:])
                if param_number < len(fortran_doubles):
                    new_value = fortran_doubles[param_number]
                    lines[i] = f'$ double precision, parameter :: {param_name}= {new_value}\n'

        # 수정된 내용을 파일에 다시 쓰기
        with open('kinet2.inp', 'w') as file:
            file.writelines(lines)

        print(f"{file_path} 파일의 파라미터가 성공적으로 업데이트되었습니다.")

    except FileNotFoundError:
        print(f"오류: {file_path} 파일을 찾을 수 없습니다.")
    except IOError as e:
        print(f"파일 처리 중 오류가 발생했습니다: {e}")
    except Exception as e:
        print(f"예상치 못한 오류가 발생했습니다: {e}")

In [4]:
def run_preprocessor(exe_path):
    try:
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.DEVNULL,
            stderr=subprocess.DEVNULL,
            stdin=subprocess.PIPE,
            universal_newlines=True
        )

        # 첫 번째 입력으로 'kinet2.inp' 전송
        process.stdin.write('kinet2.inp\n')
        process.stdin.flush()

        # 그 후 계속해서 '.' 입력
        while process.poll() is None:
            process.stdin.write('.\n')
            process.stdin.flush()

    except:
        pass

In [5]:
def compile_fortran(name):
    compile_command = [
        "gfortran",
        "-o", name,
        "dvode_f90_m.F90",
        "zdplaskin_m.F90",
        "run_plasRxn_v2.F90",
        "bolsig_x86_64_g.dll"
    ]

    try:
        # 컴파일 명령어 실행
        result = subprocess.run(compile_command, check=True, capture_output=True, text=True)
    except subprocess.CalledProcessError as e:
        # 컴파일 중 오류가 발생하면 오류 메시지 출력
        print("컴파일 실패!")
        print("오류 메시지:")
        print(e.stderr)
    except FileNotFoundError:
        # gfortran이 설치되어 있지 않거나 PATH에 없는 경우
        print("gfortran을 찾을 수 없습니다. gfortran이 설치되어 있고 시스템 PATH에 추가되어 있는지 확인하세요.")

In [6]:
def run_executable(exe_path):
    try:
        # 실행 파일 실행
        process = subprocess.Popen(
            exe_path,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            universal_newlines=True,
            bufsize=1
        )

        # 실시간으로 출력 읽기
        while True:
            output = process.stdout.readline()
            if "ZDPlasKin INFO: the density of species not configured for BOLSIG+ solver exceeds 1.00D+00" in output:
                print("지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.")
                process.send_signal(signal.SIGTERM)
                break
            if "PRESS ENTER TO EXIT" in output:
                print("지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.")
                process.send_signal(signal.SIGTERM)
                break
            #if output:
                #print(output.strip())

        # 프로세스 종료 대기
        return_code = process.wait()

    except:
        pass

In [7]:
class GaussianProcess:
    def __init__(self,kernel, noise=1e-8):
        self.kernel = kernel
        self.noise = noise
    
    def fit(self, X, y):
        self.X_train = X
        self.y_train = y
        self.K = self.kernel(self.X_train, self.X_train) + self.noise * np.eye(len(self.X_train))
        self.L = np.linalg.cholesky(self.K)
        self.alpha = np.linalg.solve(self.L.T, np.linalg.solve(self.L, self.y_train))

    def predict(self, X_test):
        K_s  = self.kernel(self.X_train, X_test)
        K_ss = self.kernel(X_test, X_test)

        Lk = np.linalg.solve(self.L, K_s)
        mu = K_s.T.dot(self.alpha)
        s2 = np.diag(K_ss) - np.sum(Lk**2, axis=0)
        return mu, s2
    
def rbf_kernel(X1, X2, l=1.0, sigma_f=1.0):
    sqdist = np.sum(X1**2, 1).reshape(-1,1) + np.sum(X2**2, 1) - 2 * np.dot(X1, X2.T)
    return sigma_f**2 * np.exp(-0.5 / l**2 * sqdist)

def expected_improvement(X, X_sample, gp, y_sample, xi=0.01):
    mu, sigma = gp.predict(X)
    mu_sample = gp.predict(X_sample)[0]
    
    sigma = sigma.reshape(-1, 1)
    mu_sample_opt = np.max(mu_sample)
    
    with np.errstate(divide='warn'):
        imp = mu - mu_sample_opt - xi
        Z = imp / sigma
        ei = imp * norm.cdf(Z) + sigma * norm.pdf(Z)
        ei[sigma == 0.0] = 0.0
        
    return ei

def propose_location(acquisition, X_sample, Y_sample, gp, bounds, n_restarts=25):
    dim = X_sample.shape[1]
    min_val = 1
    min_x = None
    
    def min_obj(X):
        return -acquisition(X.reshape(1, -1), X_sample, gp, Y_sample).ravel()
    
    for x0 in np.random.uniform(bounds[:, 0], bounds[:, 1], size=(n_restarts, dim)):
        res = minimize(min_obj, x0=x0, bounds=bounds, method='L-BFGS-B')
        if res.fun < min_val:
            min_val = res.fun
            min_x = res.x
            
    return min_x.reshape(1, -1)

In [8]:
# 메인 최적화 루프
def bayesian_optimization(n_iters, objective_function, bounds):
    dim = bounds.shape[0]
    X_sample = np.random.uniform(bounds[:, 0], bounds[:, 1], size=(10, dim))
    Y_sample = objective_function(X_sample)
    
    gp = GaussianProcess(kernel=rbf_kernel)
    
    for i in range(n_iters):
        gp.fit(X_sample, Y_sample)
        
        # 다음 샘플링 위치 제안
        next_sample = propose_location(expected_improvement, X_sample, Y_sample, gp, bounds)
        
        # 새 샘플에 대한 목적 함수 계산
        next_sample_value = objective_function(next_sample)
        
        # 샘플 데이터에 새 포인트 추가
        X_sample = np.vstack((X_sample, next_sample))
        Y_sample = np.concatenate([Y_sample, next_sample_value])
        
        print(f"Iteration {i+1}/{n_iters}, Best error: {Y_sample.min()}")
    
    with open('best_para.txt', 'w') as f:
        f.write(X_sample[Y_sample.argmin()])
    return X_sample[Y_sample.argmin()]

In [9]:
# 예시 목적 함수 (실제 모델을 여기에 연결해야 함)
def objective_function(X):
    f = np.ones(19)
    err_arr = []
    for i in range(len(X)):
        fi = f * 10 ** X[i]
        fortran_doubles = list(map(to_fortran_double, fi))

        # parameter update
        file_path = 'kinet.inp'
        update_kinet_inp(file_path, fortran_doubles)

        # zdplaskin.F90 생성
        exe_processor = r'preprocessor.exe'
        run_preprocessor(exe_processor)

        # 실행파일 생성
        exe_path="run_plasRxn_v3.exe"
        compile_fortran(exe_path)

        # 시뮬레이션 실행
        run_executable(exe_path)

        # Read raw data
        conditions = []
        with open('qt_conditions_list.txt','r') as file:
            for line in file:        
                line = line.strip()
                line = line[2:]
                conditions.append(line)
            file.close()

        species = []
        with open('qt_species_list.txt','r') as file:
            i = 0
            for line in file:        
                line = line.strip()
                if i < 9:
                    line = line[2:]
                else:
                    line = line[3:]
                species.append(line)
                i += 1
            file.close()

        reactions = []
        with open('qt_reactions_list.txt','r') as file:
            for line in file:        
                line = line.strip()
                line = line[2:]
                reactions.append(line)
            file.close()

        df_cd = pd.read_csv('qt_conditions.txt', sep=r'\s+', header=0, names=['Time [s]']+conditions)
        df_sp = pd.read_csv('qt_densities.txt', sep=r'\s+', header=0, names=['Time [s]']+species)
        df_rx = pd.read_csv('qt_rates.txt', sep=r'\s+', header=0, names=['Time [s]']+reactions)

        CH4 = (df_sp['CH4'] + df_sp['CH4(V13)'] + df_sp['CH4(V24)'])/const.N_A*16
        C2H6 = (df_sp['C2H6'] + df_sp['C2H6(V13)'] + df_sp['C2H6(V24)'])/const.N_A*30
        C2H4 = (df_sp['C2H4'] + df_sp['C2H4(V1)'] + df_sp['C2H4(V2)'])/const.N_A*28
        C2H2 = (df_sp['C2H2'] + df_sp['C2H2(V13)'] + df_sp['C2H2(V2)'] + df_sp['C2H2(V5)'])/const.N_A*26
        C3H8 = (df_sp['C3H8'] + df_sp['C3H8(V1)'] + df_sp['C3H8(V2)'])/const.N_A*44
        C3H6 = (df_sp['C3H6'] + df_sp['C3H6(V)'])/const.N_A*42
        C2H = df_sp['C2H']/const.N_A*25
        CH = df_sp['CH']/const.N_A*13
        C3H7 = df_sp['C3H7']/const.N_A*43
        C3H5 = df_sp['C3H5']/const.N_A*41
        CH3 = df_sp['CH3']/const.N_A*15
        CH2 = df_sp['CH2']/const.N_A*14
        C2H3 = df_sp['C2H3']/const.N_A*27
        C4H9 = df_sp['C4H9']/const.N_A*57
        C2H5 = df_sp['C2H5']/const.N_A*29
        C3H4 = df_sp['C3H4']/const.N_A*40
        C5H12 = df_sp['C5H12']/const.N_A*72
        C4H10 = df_sp['C4H9H']/const.N_A*58

        del_CH4 = CH4.iloc[0] - CH4.iloc[-1]
        CH4_conv_cal = del_CH4/CH4.iloc[0] * 100
        total_CH = C2H6.iloc[-1] + C2H4.iloc[-1] + C2H2.iloc[-1] + C3H8.iloc[-1] + C3H6.iloc[-1] + C2H.iloc[-1] + CH.iloc[-1] + C3H7.iloc[-1] + C3H5.iloc[-1] + CH3.iloc[-1] + CH2.iloc[-1] + C2H3.iloc[-1] + C4H9.iloc[-1] + C2H5.iloc[-1] + C3H4.iloc[-1] + C5H12.iloc[-1] + C4H10.iloc[-1]
        C2H6_sel_cal = C2H6.iloc[-1]/total_CH * 100
        C2H4_sel_cal = C2H4.iloc[-1]/total_CH * 100
        C2H2_sel_cal = C2H2.iloc[-1]/total_CH * 100
        C3H8_sel_cal = C3H8.iloc[-1]/total_CH * 100
        C3H6_sel_cal = C3H6.iloc[-1]/total_CH * 100
        C2H_sel_cal = C2H.iloc[-1]/total_CH * 100
        CH_sel_cal = CH.iloc[-1]/total_CH * 100
        C3H7_sel_cal = C3H7.iloc[-1]/total_CH * 100
        C3H5_sel_cal = C3H5.iloc[-1]/total_CH * 100
        CH3_sel_cal = CH3.iloc[-1]/total_CH * 100
        CH2_sel_cal = CH2.iloc[-1]/total_CH * 100
        C2H3_sel_cal = C2H3.iloc[-1]/total_CH * 100
        C4H9_sel_cal = C4H9.iloc[-1]/total_CH * 100
        C2H5_sel_cal = C2H5.iloc[-1]/total_CH * 100
        C3H4_sel_cal = C3H4.iloc[-1]/total_CH * 100
        C5H12_sel_cal = C5H12.iloc[-1]/total_CH * 100
        C4H10_sel_cal = C4H10.iloc[-1]/total_CH * 100    

        # error 계산
        CH4_conv_exp = 20
        C2H6_sel_exp = 40
        C2H4_sel_exp = 22
        C2H2_sel_exp = 17
        C3H8_sel_exp = 5
        C3H6_sel_exp = 10
        C4H10_sel_exp = 6
        C2H_sel_exp = 0
        CH_sel_exp = 0
        C3H7_sel_exp = 0
        C3H5_sel_exp = 0
        CH3_sel_exp = 0
        CH2_sel_exp = 0
        C2H3_sel_exp = 0
        C4H9_sel_exp = 0
        C2H5_sel_exp = 0
        C3H4_sel_exp = 0
        C5H12_sel_exp = 0
        err = 0
        err += (CH4_conv_cal - CH4_conv_exp)**2 * 10
        err += (C2H6_sel_cal - C2H6_sel_exp)**2
        err += (C2H4_sel_cal - C2H4_sel_exp)**2
        err += (C2H2_sel_cal - C2H2_sel_exp)**2
        err += (C3H8_sel_cal - C3H8_sel_exp)**2
        err += (C3H6_sel_cal - C3H6_sel_exp)**2
        err += (C4H10_sel_cal - C4H10_sel_exp)**2
        err += (C2H_sel_cal - C2H_sel_exp)**2
        err += (CH_sel_cal - CH_sel_exp)**2
        err += (C3H7_sel_cal - C3H7_sel_exp)**2
        err += (C3H5_sel_cal - C3H5_sel_exp)**2
        err += (CH3_sel_cal - CH3_sel_exp)**2
        err += (CH2_sel_cal - CH2_sel_exp)**2
        err += (C2H3_sel_cal - C2H3_sel_exp)**2
        err += (C4H9_sel_cal - C4H9_sel_exp)**2
        err += (C2H5_sel_cal - C2H5_sel_exp)**2
        err += (C3H4_sel_cal - C3H4_sel_exp)**2
        err += (C5H12_sel_cal - C5H12_sel_exp)**2
        err_arr.append(err)
    return err_arr  # 간단한 예시, 실제로는 모델의 에러를 반환해야 함

In [10]:
best = []

In [11]:
# 최적화 실행
bounds = np.array([[0, 1]] * 38)  # 각 파라미터의 범위를 -10에서 10으로 설정
best_params = bayesian_optimization(1000, objective_function, bounds)
best = best_params
print("Best parameters found:", best_params)
print("Best error:", objective_function(best_params.reshape(1, -1))[0])

kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
Iteration 1/1000, Best error: 4217.090141385867
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
Iteration 2/1000, Best error: 4217.090141385867
kinet.inp 파일의 파라미터가 성공적으로 업데이트되었습니다.
지정된 문자열이 감지되었습니다. 프로그램을 종료합니다.
Iteration 3/1000, Be

KeyboardInterrupt: 

In [12]:
print("Best parameters found:", best_params)
print("Best error:", objective_function(best_params.reshape(1, -1))[0])

Best parameters found: [ 3.11547284  1.59629022 -2.07221363 -1.21170692  0.8096885   0.17256316
 -0.09055984 -1.92959856  2.96405621 -3.9160086  -0.00844079 -0.05317957
 -2.43293842  0.69529757 -0.62474432 -1.9562296  -3.22592067 -0.53126177
 -3.23064137  3.85559582 -0.62607182  1.04457316 -3.38972344 -3.94622671
 -1.58709388  2.44671797  2.39676737 -2.18094136 -3.63083483  2.15826075
  3.64658776 -0.07814103  3.63106273  0.35848401  0.7087804  -0.05970387
 -3.68404644 -3.00742111 -3.76087901 -1.03186482  1.71581181 -1.72610961
  1.9005566  -0.09534281 -1.51477164  1.74251698  2.48760574 -3.32908055
  3.08914097  3.41412905  2.69545855  0.9370646  -3.90492266 -0.76497733
  1.32524489 -1.38273259 -0.27191811 -2.62281439  0.23665241  1.98902383
  0.41041556 -1.6408071  -3.82663258 -3.87301713  3.10079389  0.36640353
 -1.6480377  -2.01352143 -3.85478662  1.74716724 -1.04468837 -3.87082409
 -2.80650689  3.15454188 -3.06505711 -3.99058733  1.10139173  3.73522622
  3.25452195  2.51221957  1.